In [1]:
!pip install bertopic accelerate bitsandbytes xformers adjustText

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 88.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.3 MB/s eta 0:00:00
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━

In [2]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.0.0
    Uninstalling Werkzeug-3.0.0:
      Successfully uninstalled Werkzeug-3.0.0


In [3]:
import os
import pandas as pd
from tqdm import tqdm
from huggingface_hub import notebook_login
from torch import cuda
from torch import bfloat16
import transformers
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration
from bertopic import BERTopic

from dash import Dash, dcc, html
import plotly.express as px
from base64 import b64encode
import io

In [5]:
current_path = os.getcwd()
current_path = current_path.split('movie-pre-buzz-to-box-office')
path = current_path[0] + '/movie-pre-buzz-to-box-office'
path

'/content/movie-pre-buzz-to-box-office'

In [6]:
notebook_login()

In [7]:
model_id = 'meta-llama/Llama-2-13b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

print(device)

cuda:0


In [8]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [9]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [10]:
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

In [11]:
prompt = "How to make social media posts engaging?"
res = generator(prompt)
print(res[0]["generated_text"])

How to make social media posts engaging?

1. Use visuals: Incorporate images, videos, or infographics into your posts to make them more visually appealing and engaging.
2. Tell a story: Share a personal anecdote or a customer success story to connect with your audience on an emotional level.
3. Ask questions: Encourage your followers to participate by asking questions or requesting their feedback.
4. Use humor: Add a touch of humor to your posts to make them more enjoyable and shareable.
5. Be authentic: Be true to your brand voice and personality, and don't be afraid to show your human side.
6. Keep it concise: Make sure your posts are easy to read and understand, and avoid using jargon or overly technical language.
7. Use hashtags: Research relevant hashtags and include them in your posts to increase visibility and reach a wider audience.
8. Time your posts strategically: Post at times when your audience is most active and engaged, such as during peak hours or during events.
9. Exper

In [12]:
system_prompt = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for labeling topics.
<</SYS>>
"""

example_prompt = """
I have a topic that contains the following documents:
- Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
- Meat, but especially beef, is the word food in terms of emissions.
- Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

The topic is described by the following keywords: 'meat, beef, eat, eating, emissions, steak, food, health, processed, chicken'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.

[/INST] Environmental impacts of eating meat
"""

main_prompt = """
[INST]
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.
[/INST]
"""

prompt = system_prompt + example_prompt + main_prompt
print(prompt)


<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for labeling topics.
<</SYS>>

I have a topic that contains the following documents:
- Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
- Meat, but especially beef, is the word food in terms of emissions.
- Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

The topic is described by the following keywords: 'meat, beef, eat, eating, emissions, steak, food, health, processed, chicken'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.

[/INST] Environmental impacts of eating meat

[INST]
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information

In [27]:
generated_description = pd.read_csv(path+'/Data/Processed/ProcessedVideoDescription.csv',  sep='|', lineterminator='%')
video_description = pd.read_csv(path+'/Data/Video Description/videoDescription.csv')

video_description = video_description[['videoId', 'title']]
generated_description = generated_description.merge(video_description, left_on='VideoId', right_on='videoId', how='left')

generated_description['CreatedTime'] = pd.to_datetime(generated_description['CreatedTime'])
generated_description = generated_description.sort_values('CreatedTime', ascending=False).drop_duplicates('VideoId', keep='first').reset_index(drop=True)

descriptions = generated_description['OpenAIGeneratedDescription']
title = generated_description['title']

generated_description

,VideoId,Code,Prompt,OpenAIGeneratedDescription,CreatedTime,videoId,title
0,09zYzvKkCOA,200,Create a humanized description of a movie trai...,Get ready for a gripping and thought-provoking...,2023-10-08 02:59:23.709760,09zYzvKkCOA,Three Christs - Official Trailer I HD I IFC Films
1,Na6gA1RehsU,200,Create a humanized description of a movie trai...,TÁR is a captivating and visually stunning fil...,2023-10-08 02:58:40.087773,Na6gA1RehsU,TÁR - Official Trailer [HD] - In Select Theate...
2,gSMxBLlA8qY,200,Create a humanized description of a movie trai...,"In the upcoming film ""Richard Jewell,"" directe...",2023-10-08 02:57:50.413461,gSMxBLlA8qY,RICHARD JEWELL - Official Trailer [HD]
3,tSkv_sEyPSc,200,Create a humanized description of a movie trai...,"In the movie trailer for ""Mending the Line,"" w...",2023-10-08 02:57:08.438921,tSkv_sEyPSc,MENDING THE LINE | Official Trailer
4,b6qSGKN42Gc,200,Create a humanized description of a movie trai...,Get ready for a heart-stopping story that hits...,2023-10-08 02:56:23.565856,b6qSGKN42Gc,SILO - Official Trailer - Oscilloscope Laborat...
...,...,...,...,...,...,...,...
549,WuClROtJCaw,200,Create a humanized description of a movie trai...,"In this beautiful and captivating movie, we wi...",2023-10-07 18:55:16.813238,WuClROtJCaw,ANOTHER ROUND Trailer [HD] Mongrel Media
550,uYPbbksJxIg,200,Create a humanized description of a movie trai...,Get ready for a thrilling journey through hist...,2023-10-07 18:54:24.794990,uYPbbksJxIg,Oppenheimer | New Trailer
551,P_dfc0iqmig,200,Create a humanized description of a movie trai...,Hustlers is a captivating film based on a true...,2023-10-07 18:53:27.765114,P_dfc0iqmig,Hustlers | Official Trailer [HD] | Own it Now ...
552,j5BnpaiMvO4,200,Create a humanized description of a movie trai...,"In the trailer for the movie ""Mickey and the B...",2023-10-07 18:52:30.206741,j5BnpaiMvO4,MICKEY AND THE BEAR Trailer (2019) Drama Movie


In [28]:
print(descriptions[201])

Get ready for a thrilling ride in "The Nightingale"! This powerful film, directed by Jennifer Kent and featuring an incredible cast, takes you on a journey through love, revenge, and the consequences of violence.

Set in colonial Australia in 1825, the story follows Clare, a 21-year-old Irish convict played by Aisling Franciosi. After serving her 7-year sentence, Clare is desperate to break free from her abusive master, Lieutenant Hawkins, portrayed by Sam Claflin. When British authorities fail to deliver justice for the crimes she suffers, Clare decides to take matters into her own hands.

In her quest for revenge, Clare enlists the help of Billy, a young Aboriginal tracker played by Baykali Ganambarr. Although they initially have a hostile relationship, both carrying their own traumas and distrust, their journey takes them through the rugged wilderness and forces them to find empathy for one another.

As they navigate through the treacherous terrain and the conflicts between the orig

In [29]:
embedding_model = SentenceTransformer("BAAI/bge-large-en")
embeddings = embedding_model.encode(descriptions, show_progress_bar=True)

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

In [30]:
umap_model = UMAP(n_neighbors=4, n_components=9, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=4, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [31]:
reduced_embeddings = UMAP(n_neighbors=4, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)

In [32]:
keybert = KeyBERTInspired()
mmr = MaximalMarginalRelevance(diversity=0.3)
llama2 = TextGeneration(generator, prompt=prompt)

representation_model = {
    "KeyBERT": keybert,
    "Llama2": llama2,
    "MMR": mmr,
}

In [33]:
from bertopic import BERTopic

topic_model = BERTopic(
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  # nr_topics='auto',
  representation_model=representation_model,

  top_n_words=10,
  verbose=True
)

topics, probs = topic_model.fit_transform(descriptions, embeddings)

2023-10-08 18:27:24,277 - BERTopic - Reduced dimensionality
2023-10-08 18:27:24,307 - BERTopic - Clustered reduced embeddings
100%|██████████| 34/34 [04:40<00:00,  8.25s/it]


In [34]:
iden_topics = topic_model.get_topic_info()
iden_topics.to_csv('iden_topics.csv', index=False)
iden_topics

,Topic,Count,Name,Representation,KeyBERT,Llama2,MMR,Representative_Docs
0,-1,85,-1_the_and_of_to,"[the, and, of, to, in, film, with, as, this, t...","[trailer, film, movie, journey, captivating, s...","[Movie Trailers, , , , , , , , , ]","[the, and, of, to, in, film, with, as, this, t...",[Get ready to embark on a heartfelt journey in...
1,0,113,0_the_of_trailer_and,"[the, of, trailer, and, we, to, in, with, as, ...","[trailer, film, movie, captivating, narrative,...","[Movie Trailers, , , , , , , , , ]","[the, of, trailer, and, we, to, in, with, as, ...",[Hustlers is a captivating film based on a tru...
2,1,30,1_the_and_of_trailer,"[the, and, of, trailer, to, in, faith, with, o...","[trailer, film, movie, journey, witness, provo...","[Religious Journeys and Self-Discovery, , , , ...","[the, and, of, trailer, to, in, faith, with, o...","[In this humanized movie trailer description, ..."
3,2,25,2_by_to_and_her,"[by, to, and, her, film, this, of, his, love, ...","[heartwarming, film, journey, cast, captivatin...","[Heartwarming Movie Releases, , , , , , , , , ]","[by, to, and, her, film, this, of, his, love, ...",[In a world where finding true happiness seems...
4,3,24,3_the_of_and_you,"[the, of, and, you, to, is, this, film, on, as]","[trailer, film, movie, cinematic, watch, journ...","[Movies and their themes, , , , , , , , , ]","[the, of, and, you, to, is, this, film, on, as]",[Get ready for an emotional and powerful movie...
5,4,18,4_and_to_the_you,"[and, to, the, you, on, of, this, film, with, ...","[film, ifc, films, cinematic, vod, captivated,...","[Movie Previews, , , , , , , , , ]","[and, to, the, you, on, of, this, film, with, ...",[Get ready to dive into the captivating world ...
6,5,18,5_the_and_of_trailer,"[the, and, of, trailer, to, in, as, with, we, ...","[trailer, film, movie, compelling, provoking, ...","[Movie Trailers, , , , , , , , , ]","[the, and, of, trailer, to, in, as, with, we, ...","[In the movie trailer for ""The Killing Floor"",..."
7,6,17,6_faith_and_to_the,"[faith, and, to, the, god, of, his, this, is, in]","[trailer, film, movie, journey, believe, inspi...","[Inspirational True Stories, , , , , , , , , ]","[faith, and, to, the, god, of, his, this, is, in]",[Get ready to be captivated by the heart-wrenc...
8,7,15,7_and_of_the_this,"[and, of, the, this, film, on, they, that, you...","[film, movie, heartwarming, journey, cast, fri...","[Coming-of-age films, , , , , , , , , ]","[and, of, the, this, film, on, they, that, you...",[Get ready for a thought-provoking and emotion...
9,8,14,8_her_and_the_of,"[her, and, the, of, to, on, this, elvis, music...","[trailer, cinematic, film, movie, journey, wat...","[Movie Trailers, , , , , , , , , ]","[her, and, the, of, to, on, this, elvis, music...",[Introducing the new theatrical trailer for Ca...


In [35]:
videoId = list(generated_description['videoId'])
red_embeddings = list(reduced_embeddings)
probabilities = list(probs)

data = []
for i, topic in enumerate(topics):
  data.append([videoId[i], topic, probabilities[i], red_embeddings[i][0], red_embeddings[i][1]])

data = pd.DataFrame(data, columns=['VideoId', 'Topic', 'Probaibility', 'ReducedEmbeddings_x', 'ReducedEmbeddings_y'])
data.to_csv('topic_classification.csv', index=False)
data

,VideoId,Topic,Probaibility,ReducedEmbeddings_x,ReducedEmbeddings_y
0,09zYzvKkCOA,6,0.960030,2.788041,2.322691
1,Na6gA1RehsU,0,1.000000,7.812714,3.023268
2,gSMxBLlA8qY,17,0.673917,5.310845,2.776821
3,tSkv_sEyPSc,5,0.478666,5.177978,3.813516
4,b6qSGKN42Gc,3,1.000000,2.385987,3.115065
...,...,...,...,...,...
549,WuClROtJCaw,6,1.000000,2.976135,2.231939
550,uYPbbksJxIg,18,0.591520,3.114096,4.091232
551,P_dfc0iqmig,0,1.000000,6.612958,2.591962
552,j5BnpaiMvO4,0,1.000000,8.060444,4.137704


In [36]:
topic_model.get_topic(1, full=True)["KeyBERT"]

[('trailer', 0.87884295),
 ('film', 0.84133554),
 ('movie', 0.8374326),
 ('journey', 0.8255658),
 ('witness', 0.8198749),
 ('provoking', 0.8186442),
 ('cast', 0.8181363),
 ('church', 0.8116478),
 ('faith', 0.8097112),
 ('captivating', 0.8080058)]

In [37]:
llama2_labels = [label[0][0].split("\n")[0] for label in topic_model.get_topics(full=True)["Llama2"].values()]
topic_model.set_topic_labels(llama2_labels)

In [40]:
plot = topic_model.visualize_documents(title, reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=False, custom_labels=True)
import plotly.offline as pyo
pyo.plot(plot, filename='plotly_plot_topics.html')
plot

In [54]:
iden_topics = pd.read_csv('iden_topics.csv')
topic_classification = pd.read_csv('topic_classification.csv')

In [55]:
iden_topics['Name'] = iden_topics['Llama2'].apply(lambda row: row.split(",")[0].replace("'",'').replace('[',''))
iden_topics

,Topic,Count,Name,Representation,KeyBERT,Llama2,MMR,Representative_Docs
0,-1,85,Movie Trailers,"['the', 'and', 'of', 'to', 'in', 'film', 'with...","['trailer', 'film', 'movie', 'journey', 'capti...","['Movie Trailers', '', '', '', '', '', '', '',...","['the', 'and', 'of', 'to', 'in', 'film', 'with...",['Get ready to embark on a heartfelt journey i...
1,0,113,Movie Trailers,"['the', 'of', 'trailer', 'and', 'we', 'to', 'i...","['trailer', 'film', 'movie', 'captivating', 'n...","['Movie Trailers', '', '', '', '', '', '', '',...","['the', 'of', 'trailer', 'and', 'we', 'to', 'i...",['Hustlers is a captivating film based on a tr...
2,1,30,Religious Journeys and Self-Discovery,"['the', 'and', 'of', 'trailer', 'to', 'in', 'f...","['trailer', 'film', 'movie', 'journey', 'witne...","['Religious Journeys and Self-Discovery', '', ...","['the', 'and', 'of', 'trailer', 'to', 'in', 'f...","['In this humanized movie trailer description,..."
3,2,25,Heartwarming Movie Releases,"['by', 'to', 'and', 'her', 'film', 'this', 'of...","['heartwarming', 'film', 'journey', 'cast', 'c...","['Heartwarming Movie Releases', '', '', '', ''...","['by', 'to', 'and', 'her', 'film', 'this', 'of...",['In a world where finding true happiness seem...
4,3,24,Movies and their themes,"['the', 'of', 'and', 'you', 'to', 'is', 'this'...","['trailer', 'film', 'movie', 'cinematic', 'wat...","['Movies and their themes', '', '', '', '', ''...","['the', 'of', 'and', 'you', 'to', 'is', 'this'...",['Get ready for an emotional and powerful movi...
5,4,18,Movie Previews,"['and', 'to', 'the', 'you', 'on', 'of', 'this'...","['film', 'ifc', 'films', 'cinematic', 'vod', '...","['Movie Previews', '', '', '', '', '', '', '',...","['and', 'to', 'the', 'you', 'on', 'of', 'this'...","[""Get ready to dive into the captivating world..."
6,5,18,Movie Trailers,"['the', 'and', 'of', 'trailer', 'to', 'in', 'a...","['trailer', 'film', 'movie', 'compelling', 'pr...","['Movie Trailers', '', '', '', '', '', '', '',...","['the', 'and', 'of', 'trailer', 'to', 'in', 'a...","['In the movie trailer for ""The Killing Floor""..."
7,6,17,Inspirational True Stories,"['faith', 'and', 'to', 'the', 'god', 'of', 'hi...","['trailer', 'film', 'movie', 'journey', 'belie...","['Inspirational True Stories', '', '', '', '',...","['faith', 'and', 'to', 'the', 'god', 'of', 'hi...","[""Get ready to be captivated by the heart-wren..."
8,7,15,Coming-of-age films,"['and', 'of', 'the', 'this', 'film', 'on', 'th...","['film', 'movie', 'heartwarming', 'journey', '...","['Coming-of-age films', '', '', '', '', '', ''...","['and', 'of', 'the', 'this', 'film', 'on', 'th...",['Get ready for a thought-provoking and emotio...
9,8,14,Movie Trailers,"['her', 'and', 'the', 'of', 'to', 'on', 'this'...","['trailer', 'cinematic', 'film', 'movie', 'jou...","['Movie Trailers', '', '', '', '', '', '', '',...","['her', 'and', 'the', 'of', 'to', 'on', 'this'...","[""Introducing the new theatrical trailer for C..."


In [76]:
dict_rename = {22: 'Emotional Journeys',
               5: 'Uncovering Painful Truths',
               8: 'Empowering Stories of Resilience',
               12: 'Love and Loss Emotional Journeys',
               18: 'Emotional Journeys Through Life and History'}

In [77]:
names = list(iden_topics['Name'])
for topic, nam in zip(dict_rename.keys(), dict_rename.values()):
  names[topic+1] = nam

names

['Movie Trailers',
 'Movie Trailers',
 'Religious Journeys and Self-Discovery',
 'Heartwarming Movie Releases',
 'Movies and their themes',
 'Movie Previews',
 'Uncovering Painful Truths',
 'Inspirational True Stories',
 'Coming-of-age films',
 'Empowering Stories of Resilience',
 'Movies and TV shows based on trailers',
 'Heartwarming Friendship Stories',
 'Biographies and Historical Dramas',
 'Love and Loss Emotional Journeys',
 'Romantic Dramas',
 'Movies and Truth',
 'Mystery and Suspense Movies',
 'Legendary Athletes in Film',
 'True Crime Dramas',
 'Emotional Journeys Through Life and History',
 'Movie Synopses',
 'True stories of historical figures',
 'Horror and Ghost Stories',
 'Emotional Journeys',
 'Movie Trailers',
 'Movies about Running and Personal Growth',
 'Movies and their stars',
 'Romantic Comedies',
 'Mafia and Organized Crime',
 'Movie Trailers',
 'Movie Trailers',
 'Innovators and Their Dreams',
 'Movies and Literature',
 'Adoption and Family Dynamics']

In [78]:
topic_model.set_topic_labels(names)

In [79]:
plot = topic_model.visualize_documents(title, reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=False, custom_labels=True)
import plotly.offline as pyo
pyo.plot(plot, filename='plotly_plot_topics.html')
plot

In [80]:
iden_topics = topic_model.get_topic_info()
iden_topics.to_csv('iden_topics.csv', index=False)
iden_topics

,Topic,Count,Name,CustomName,Representation,KeyBERT,Llama2,MMR,Representative_Docs
0,-1,85,-1_the_and_of_to,Movie Trailers,"[the, and, of, to, in, film, with, as, this, t...","[trailer, film, movie, journey, captivating, s...","[Movie Trailers, , , , , , , , , ]","[the, and, of, to, in, film, with, as, this, t...",[Get ready to embark on a heartfelt journey in...
1,0,113,0_the_of_trailer_and,Movie Trailers,"[the, of, trailer, and, we, to, in, with, as, ...","[trailer, film, movie, captivating, narrative,...","[Movie Trailers, , , , , , , , , ]","[the, of, trailer, and, we, to, in, with, as, ...",[Hustlers is a captivating film based on a tru...
2,1,30,1_the_and_of_trailer,Religious Journeys and Self-Discovery,"[the, and, of, trailer, to, in, faith, with, o...","[trailer, film, movie, journey, witness, provo...","[Religious Journeys and Self-Discovery, , , , ...","[the, and, of, trailer, to, in, faith, with, o...","[In this humanized movie trailer description, ..."
3,2,25,2_by_to_and_her,Heartwarming Movie Releases,"[by, to, and, her, film, this, of, his, love, ...","[heartwarming, film, journey, cast, captivatin...","[Heartwarming Movie Releases, , , , , , , , , ]","[by, to, and, her, film, this, of, his, love, ...",[In a world where finding true happiness seems...
4,3,24,3_the_of_and_you,Movies and their themes,"[the, of, and, you, to, is, this, film, on, as]","[trailer, film, movie, cinematic, watch, journ...","[Movies and their themes, , , , , , , , , ]","[the, of, and, you, to, is, this, film, on, as]",[Get ready for an emotional and powerful movie...
5,4,18,4_and_to_the_you,Movie Previews,"[and, to, the, you, on, of, this, film, with, ...","[film, ifc, films, cinematic, vod, captivated,...","[Movie Previews, , , , , , , , , ]","[and, to, the, you, on, of, this, film, with, ...",[Get ready to dive into the captivating world ...
6,5,18,5_the_and_of_trailer,Uncovering Painful Truths,"[the, and, of, trailer, to, in, as, with, we, ...","[trailer, film, movie, compelling, provoking, ...","[Movie Trailers, , , , , , , , , ]","[the, and, of, trailer, to, in, as, with, we, ...","[In the movie trailer for ""The Killing Floor"",..."
7,6,17,6_faith_and_to_the,Inspirational True Stories,"[faith, and, to, the, god, of, his, this, is, in]","[trailer, film, movie, journey, believe, inspi...","[Inspirational True Stories, , , , , , , , , ]","[faith, and, to, the, god, of, his, this, is, in]",[Get ready to be captivated by the heart-wrenc...
8,7,15,7_and_of_the_this,Coming-of-age films,"[and, of, the, this, film, on, they, that, you...","[film, movie, heartwarming, journey, cast, fri...","[Coming-of-age films, , , , , , , , , ]","[and, of, the, this, film, on, they, that, you...",[Get ready for a thought-provoking and emotion...
9,8,14,8_her_and_the_of,Empowering Stories of Resilience,"[her, and, the, of, to, on, this, elvis, music...","[trailer, cinematic, film, movie, journey, wat...","[Movie Trailers, , , , , , , , , ]","[her, and, the, of, to, on, this, elvis, music...",[Introducing the new theatrical trailer for Ca...


In [82]:
videoId = list(generated_description['videoId'])
red_embeddings = list(reduced_embeddings)
probabilities = list(probs)
topics = list(topic_model.topics_)

data = []
for i, topic in enumerate(topics):
  data.append([videoId[i], topic, probabilities[i], red_embeddings[i][0], red_embeddings[i][1]])

data = pd.DataFrame(data, columns=['VideoId', 'Topic', 'Probaibility', 'ReducedEmbeddings_x', 'ReducedEmbeddings_y'])
data.to_csv('topic_classification.csv', index=False)
data

,VideoId,Topic,Probaibility,ReducedEmbeddings_x,ReducedEmbeddings_y
0,09zYzvKkCOA,6,0.960030,2.788041,2.322691
1,Na6gA1RehsU,0,1.000000,7.812714,3.023268
2,gSMxBLlA8qY,17,0.673917,5.310845,2.776821
3,tSkv_sEyPSc,5,0.478666,5.177978,3.813516
4,b6qSGKN42Gc,3,1.000000,2.385987,3.115065
...,...,...,...,...,...
549,WuClROtJCaw,6,1.000000,2.976135,2.231939
550,uYPbbksJxIg,18,0.591520,3.114096,4.091232
551,P_dfc0iqmig,0,1.000000,6.612958,2.591962
552,j5BnpaiMvO4,0,1.000000,8.060444,4.137704
